# Exercise 2: solving a bungee jump differential equation

Many problems in physics can be modelled with differential equations. It is also the case that many of these cannot be solved analytically! In this exercise you will learn how to solve them using numerical methods, culminating in solving the non-linear dynamics of a bungee jump as using a built-in ODE solver.

<div class="alert alert-block alert-info">
<h3>This exercise will cover:</h3> 
</div>

 * Using numerical approachs to approximately solve differential equations, a common application of computers in physical sciences <br>(finite element analysis, climate modelling, any kind of computer simulation) 
 * Plotting variables against each other with `matplotlib`
 * Making use of a library functions (`solve_ipv` from `scipy`)
 * Using python functions as mathematical functions 
 * Using python functions to reduce work by doing the same thing with different starting values (a *proceedure*)
 * Building up more complicated function out of simpler ones
 
 <div class="alert alert-block alert-info"></div>
 

## Example: Exponential decay

As an example, lets consider exponential decay.  To make the question concrete, let's take the example of a sample of a radioisotope with a lifetime $\tau$.  The number of nuclei remaining at a given time $t$ is $N$, and we know that $N(t)$ obeys: 

$$ \frac{\mathrm{d}N}{\mathrm{d}t} = -\frac{1}{\tau} N  \qquad \Longrightarrow \qquad N = N_0 e^{-t/\tau}$$

In this case we have the analytic solution and could just plot it. But how could we approach the problem if that wasn't true?  Well, the differential equation is essentially just a rule for telling us how $N$ changes after a small step in time:

$$ \mathrm{d}N = -\frac{1}{\tau} N \mathrm{d}t $$

So we can approximate that by *discretising* the problem; breaking it up into a large number of discrete (not discreet) time steps separated by a small interval, $\Delta t$.

$$ \begin{aligned} 
t \quad \longrightarrow & \quad t_0, \quad t_1, ~\ldots, ~t_{i-1}, \quad t_i, \quad t_{i+1}, ~ \ldots \\
%\Big( t & =     & 0,   & \Delta t, & \ldots, & (n-1)\Delta t, & n\Delta t, & \ldots & \Big)  
N(t) \quad \longrightarrow & \quad N_0,  ~ N_1, ~\ldots, ~N_{i-1}, ~N_i,  N_{i+1}, ~ \ldots 
\end{aligned}$$


Assuming we are given $N_i$, the number of nuclei left at $t_i$, we can say, (assuming that $\Delta t$ is small): 

$$[\Delta N]_{\,i} = N_{i+1} - N_{i} = -\frac{\Delta t}{\tau} \cdot N_{i}$$

which is just the difference equation evaluated at time $t_i$.  Rearranging this, we get a rule for calculating $N_{i+1}$ in terms of just $N_{i}$:

$$
N_{i+1} =  (1-\Delta t/\tau) \cdot N_{i} 
$$

so, for instance, if $\Delta t /\tau = 0.01$, $N$ would reduce by a constant factor of $0.99$ at each time step.  





<div class="alert alert-block alert-warning">
<h3>Check your understanding</h3>
</div>
    
1. Try turning the following first-order differential equations into difference equations:
 *   $\dfrac{\mathrm{d}Z}{\mathrm{d}t} = -kZ^2$ into a difference equation for $Z_{i+1}$
 *   $\dfrac{\mathrm{d}f}{\mathrm{d}t} = \dfrac{1}{t}$ into a difference equation for $f_{i+1}$ (*Remember to discretise $t_i$.*)
 *   $a \dfrac{\mathrm{d}y}{\mathrm{d}t} + by + c  = 0$, into a difference equation for $y_{i+1}$ (*Start by moving everything except the derivative to the RHS*) <br><br>    

2. Why is it important that the step size is small?  
 *Hint: think about comparing two steps of size $0.01\tau$ to one of size $0.02\tau$*

<!--1.  **[Hard]** We defined $[\Delta N]_{\,i} = N_{i+1} - N_{i}$, which is called the *forward difference*.  The alternative *backward difference* is $[\Delta N]^\mathrm{BWD}_{\,i} = N_{i} - N_{i-1}$. 
 * Can we use the backward difference $[\Delta N]^\mathrm{BWD}_{\,i}$ to create a rule for calcualting $N_{i+1}$ from $N_i$ (or equivalently $N_i$ from $N_{i-1}$ from $[\Delta N]^\mathrm{BWD}_{\,i}$) in the same way, or is there a problem?
 * Ignoring practicalities, and just considering the maths: is there a reason to prefer one definition over the other?  Is there any other way to define the difference?
 <br><br> -->

<div class="alert alert-block alert-warning"> </div>



### Implementing the difference equation in code

Using the forward  difference to 'solve' a differential equation has a name, the *Euler method* (so clearly it predates electronic computers). Let's break it down into steps:
1. We have some points on a function to find $y_i$, at a set of discrete co-ordinate points $t_i$;
1. The function to be approximated $y$ has a known rate of change, given by $\dot{y} (y,t)$  
 *  For compactness, it is common to use $\dot{y} = \mathrm{d}y / \mathrm{d}t$. It also reminds us that as far as the computer is concerned this is just another number.
 *  In the decay example $\dot{N}$ has no explicit dependence on $t$. But the Euler method would still work if it did,because we can easily calculate $t_{i+1} = t_i + \Delta t$ 
1. We can find $y_{n+1}$ by using the approximation that $y_{i+1} = y_i + \Delta t \cdot \dot{y}(y_i,t_i)$, where $\Delta t$ is the desired time-step;
1. Given a starting point $y_0$ at $t_0$ we can use this to iteratively calculate each $y_i$ in turn.

In this example of a radioisotope decay, we used $N_i$ (and therefore $\dot{N}_i$) instead of $y_i$ and $\dot{y}_i$. Of course it does not matter what letters are actually used.  These *dependent variables* are the functions we take the derivatives of, and are usually what we are trying to find out.  Conversely $t$ is the *independent variable*, which is the thing we do derivatives in.  The time $t$ is by far the most common independent variable for an ODE, but if you were looking at temperature along a bar it might be a spatial co-ordinate $x$.  Spatial co-ordiantes could be dependent variables too, as in the problems we will solve below.  What matters is not what the variable represents, but the role they have (where they appear) in the derivatives.   

<div class="alert alert-block alert-info">
The <a href="https://en.wikipedia.org/wiki/Euler_method">Wikipedia page</a> has more detail of the Euler method, including a derivation and an informal description of why the algorithm works. <br>(The page also notes that the Euler method is also namechecked in the 2016's film <i>Hidden Figures</i>.  In reality, the NASA <a href="https://ntrs.nasa.gov/search.jsp?R=19980227091">Technical Report</a> that Katherine Johnson authored doesen't use the Euler method, although it does make use of a iterative numerical method. Euler is just used in the film since it's maths-famous.) 
</div>

Now, how to implement this in Python?  In practice we will want to plot the output so the time points $t_i$ will be a `list`, or more likely a `ndarray` since we will probably want to use numpy at some point. Likewise, the $y_i$,  would also be a `ndarray`. 
> An alternative would be to have a 2D array, representing an 1D array of $(t_i,N_i)$ pairs. In my experience it's quite common to have a single time axis and plot different functions along it, so I'd more often choose not to pair up the $N_i$ with $t_i$, but either approach would work.

The 'rule' for updating $N_i$ uses the time derivative $\dot{N}(N,t)$ (which is what we're trying to solve).  In general, the Euler method is independent of what that differential equation actually is, so let's make that a separate python function that we can call (and change if we want to look at a different problem). So write a function to to calculate $\dot{N}$:

In [1]:
def expo_decay_ode(N, lifetime = 10):
    """
    Calculates the time derivative for exponential decay ODE. 
    In this case we don't need to pass in a time as the equation only depends on the current N. 
    We would probably expect to be given a value of the lifetime, but let's add a default (=10)
    """
    return  - N / lifetime


# CHECK: if N=100, dN/dt should be -10. 
CHECK_VALUE = -10. 
assert expo_decay_ode(100) == CHECK_VALUE, f"{expo_decay_ode(100)} is not equal to {CHECK_VALUE}"

print("Check completes ok")


Check completes ok


In the actual calculation we will take a value of $N_i$, calculate the forward difference $[\Delta N]_i$, then use that to calcualte $N_{i+1}$: 


In [2]:
#Python programmer's convention: The capital letters indicate that this global (can be used anywhere) variable is a constant (I should not change it anywhere else). 
#   It is just a convention though, unlike some languages Python does not stop me.
TIMESTEP_SIZE = 0.5

def step_N(Ni, timestep = TIMESTEP_SIZE):
    """Calculates N[i+1] given N[i], assuming the decay ODE"""
    
    #N_i+1 = N_i + dt * dNdt
    dN = timestep * expo_decay_ode(Ni)
    return Ni + dN   


# CHECK: if N=100 tau=10 and step=1, we should have 90 left after 1 step, and 81 after the second. 
CHECK_N1 = 90.
N1 = step_N(100, 1)
assert N1 == CHECK_N1, f"{N1} is not equal to {CHECK_N1}"

CHECK_N2 = 81.
N2 = step_N(N1, 1);
assert N2 == CHECK_N2, f"{N2} is not equal to {CHECK_N2}"

print("Check completes ok")

Check completes ok


All we need to do now is actually step through the solver and see what happens.  There are a number of ways to do this in Python, but a `for` loop is the most universal, (i.e. something very similar would be an option in most programming languages) so we'll use that for this example.  


In [3]:
import numpy as np

# First prepare a variable to hold all the time points 
# As a rule of thumb, the length should be few times longer than any time scales in the problem (the lifetime in this case)
MAX_TIME = 50
timepoint = np.arange(0, 50, TIMESTEP_SIZE)  


#Now an array to hold values of N
#I know that -1 should never show up in this system, so I can use it as a 'bad' value.  (This is not always possible)
#If I see -1 in the output thats a clear sign I've done something wrong.
N = np.full_like(timepoint, -1.)  

#Just pick 1000 as a starting number of nuclei.
N[0] = 1000
#print(N)

#loop over all the time points, starting at t[1].  At each point in time update N.
for i in range(1,len(timepoint)):
    N[i] = step_N(N[i-1])
    
    
#Display the output. A plot would be better, but this is just an example to get you started...    
print(N)
    


[1000.          950.          902.5         857.375       814.50625
  773.7809375   735.09189063  698.33729609  663.42043129  630.24940972
  598.73693924  568.80009228  540.36008766  513.34208328  487.67497912
  463.29123016  440.12666865  418.12033522  397.21431846  377.35360254
  358.48592241  340.56162629  323.53354497  307.35686773  291.98902434
  277.38957312  263.52009447  250.34408974  237.82688526  225.93554099
  214.63876394  203.90682575  193.71148446  184.02591024  174.82461472
  166.08338399  157.77921479  149.89025405  142.39574135  135.27595428
  128.51215657  122.08654874  115.9822213   110.18311024  104.67395472
   99.44025699   94.46824414   89.74483193   85.25759033   80.99471082
   76.94497528   73.09772651   69.44284019   65.97069818   62.67216327
   59.53855511   56.56162735   53.73354598   51.04686868   48.49452525
   46.06979899   43.76630904   41.57799359   39.49909391   37.52413921
   35.64793225   33.86553564   32.17225886   30.56364591   29.03546362
   27.583

## Part 1: A pendulum

In your previous courses you should have learned about simple harmonic oscillators. You may remember that a pendulum is only approximately a simple harmonic oscillator, and it's actually following a nonlinear equation. Recall that a simple mathematical pendulum without friction can be modelled by the second order ODE:

$$\frac{d^2\theta}{dt^2} + \frac{g}{l}\sin{\theta} = 0$$

where $g$ is the gravitational acceleration and $l$ is the length of the string.  The analytic solution starts by assuming that $\theta$ is small, so we can make the approximation $\sin{\theta} = \theta$ and recover a simple harmonic oscillator.   The useful thing about numerical solvers is that we don't need to make that assumption. Not only can we avoid that approximation, but we can also use numerical methods for situations where there is no analytic approximation at all.   

### The second-order ODE
Our goal is to write code to find a numerical solution for this oscillator. But, we have a new problem: the harmoni oscillator  is a *second order* ODE, meaning in this case that the equation involves the acceleration (but not the velocity).  However there is a trick for dealing with this:  We introduce the first derivative (angular velocity, $\omega$) as a new coordinate, and split the equation into two steps:    
$$\frac{d\theta}{dt}=\omega~ ; \quad \frac{d\omega}{dt}=-\frac{g}{l}\sin{\theta}.$$

You can verify for yourself that this works by taking the time derivative of the first equation and substituting in the second.

This approach of adding a new co-ordinate and splitting a second-order ODE int two *coupled* first-order ODEs is very useful computationally.  It is also very important physically: the expanded $(\theta, \omega)$ space - or more generally $(x, \dot{x})$ space - is called (dynamical) *phase space*.

<div class="alert alert-block alert-info">
You can read about the use of phase space in dynamics online (e.g. <a href="https://en.wikipedia.org/wiki/Phase_space"> on Wikipedia</a>). Its arguably the most important concept that isn't covered by most undergraduate degrees.  Phase space is central in Lagrange and Hamiltonian formulations of classical mechanics (where we typically use $p = m\dot{x}$ as the second coordinate).  Phase space is also a recognisable feature of quantum mechanics: the Heisenberg Uncertainty Principle essentially amounts to a statement that the particle being considered has a minimum size in phase space.
</div>

As with the exponential decay, 'solving' here means taking the initial state (i.e. $(\theta(t=0), \omega(t=0))$) and evolving that state to see what it is at some later point in time.  This is called an *initial value* problem, as we start from some initial value of the (two-component) state and work out the value at some later time $t$.  

>  Finding such solutions is also described as *integrating* the differential equation, since we start with some expression for some derivatives, and ends up with a solution describing the state.  However, except in very simple cases this does not correspond directly to a doing an integral, so it's a little misleading.

<div class="alert alert-block alert-success">
    <h3>Computing task : Euler solution to the non-linear oscillator</h3>
</div>

Solve the pendulum numerically, following the approach in the example.
 1. Turn the differential equations into difference equations.  
 1. Write functions to implement these difference equations. You will need to update $\omega$ based on the current value of $\theta$, and then update $\theta$ using your new value of $\omega$.  
 1. Step through time updating $\omega$ and $\theta$ at each step.  Don't forget to provide a parameter to control how big each timestep is, and think about how many timesteps you need to properly understand the solutions.  You'll also need to think about how to provide a starting state $(\theta_0,\omega_0)$, and the length of the pendulum $l$.
 1. When you have a solution, use MatPlotLib to plot $\theta$ and $\omega$ as a function of $t$, and see if they match your expectations.  Check with a demonstrator if they don't. <br>

#### Now investigate your solutions
 * Select the starting conditions for the angle to be small ($\pi/8$) and for the angular velocity to be 0. Let the length of the pendulum be $l = 1$. <br>
  Do $\theta$ and $\omega$ evolve in the way you expect for a pendulum? Try a few other starting conditions and see if it behaves the way you expect.
 * Now go back to the original small-amplitude oscillations, try changing the timestep:  $\Delta t = 0.1$, $1$, and $10$.  Can you explain what you see? 
 * Going back to a small timestep, like $0.01$, try increasing the size of the starting angle (remember to stay below $\pi/2$).  <br> 
 For a simple harmonic oscillator, increasing the starting angle just scales the amplitude of the oscillation. Can you see any deviation from this scaling behaviour?  
 
 *Hint: Solving the pendulum equation will be much faster (and much easier to maintain) if you can write a function that takes the set of parameters you are interested in, then runs the solver and draws the results.  In this appraoch this you will probably end up with functions calling funtions which is very common in larger software projects*
 
 <div class="alert alert-block alert-success"></div>



## Part 2: Better solvers

The results may not be what you expected! In particular when we try to make the step size larger, the numerical solver can become unstable: the values of $\theta$ and $\omega$ start following a very bumpy pattern, and eventually the solution "blows up" completely.  The smaller step sizes don't cause problems, but the smaller the step size, the longer it will take.  

> And if step size is *too* small, problems do start to arise due to floating point accuracy in the computer: adding small numbers to a big one results in no change to the big number.  In our case all the numbers in the problem are $\mathcal{O}(1)$, and Python uses double-precision format, which has a precision of 1 part in 10<sup>16</sup>.  However, this can become an issue if less precise formats are used and/or the problem is not carefully posed.

Ensuring stability of a numerical solution becomes even more complicated with higher-order or non-linear ODEs.  In general, these problems are not trivial to solve. Luckily, we don't have to do it ourselves.  The `scipy.integrate` module has a number of utilities for dealing with numerical problems in integration, including a number of ODE solvers. A short description of how they work is given below, but as always, refer to the [SciPy documentation](https://docs.scipy.org/doc/scipy-1.5.0/reference/generated/scipy.integrate.solve_ivp.html)  for the function for more details. The `solve_ipv` function ("solve initial value problem") is what you will need to find a solution in this case.

We first have to define a function that describes our equation that we will give to the solver. Passing functions as arguments into other functions might seem weird, but it is a powerful technique, and is even the basis of some programming paradigms.

The function that we have to provide has the signature $\mathtt{function(t,y)} = \mathbf{\dot{y}}(t,\mathbf{y})$, that is to say it takes a time co-ordinate, and a (possibly array-like) variable and returns the time derivatives. If you tackled part 1 by replicating the structure of the example, you may have a function quite similar to this.  But your function probably isn't useable with this solver. Even if it does basically the same calculation, it will fail to work if it does not use the *signature* the solver utility expects.  

> *Why is that?*  Because at this low(ish) level, the computer is extremely stupid.  It doesn't know what you are trying to do, or even whether your variable names meaning anything.  All the solver knows is that it needs to feed certain numbers into the various inputs of the function, and it will get back certain other numbers in the output.  It then follows some pre-written 'rule' about where to find the numbers it should use as the first input, and what to do with the outputs to get a new input.  It's up to you make sure the function you give it has inputs and outputs in the right place (literally: the variables car converted into locations in the memory when run) so that it abides by the same rules.  This is why `scipy`  and `numpy` documentation  is so long and dry:  its essentially a detailed contract that the library code functions are written to obey.

So the pendulum ODE, written with the necessary signature to use with SciPy is:

In [4]:
#Probably already did this...
import numpy as np

def part2_pend_ode(t, coords):
    "Function defining the pendulum ODE.  Signature as specified in scipy.integrate"
    Q, W = coords #< tuple unpacking.  
    # Here I use Q and W as theta and omega, respectively.  
    # Single-letter variable names are fine in small functions, but be more careful if you have big blocks (>10 lines) of code
    
    g = 9.81
    l = 1
    
    dQ_dt = W
    dW_dt = -g/l * np.sin(Q)
    return [dQ_dt,dW_dt]

#CHECK: If my pendulum is at positive displacement and at rest it will accelerate (dW/dt) in a -ive direction
CHECK_START1 = (0.1,0.0)
print(part2_pend_ode(0, CHECK_START1), "should be [0, <0]?")

#CHECK: If my pedulum is at zero displacement but moving in a positive direction the change in angle (dQ/dt) will be positive.
CHECK_START2 = (0.0,0.1)
print(part2_pend_ode(0, CHECK_START2), "should be [>0, 0]?")


[0.0, -0.9793658173053844] should be [0, <0]?
[0.1, -0.0] should be [>0, 0]?


With the system of ODEs defined, we can call the `solve_ipv()` function from `scipy.integrate`; this is based on a Runge-Kutta 4th order solver. The integrator has three required arguments ("parameters" in the [documentation](https://docs.scipy.org/doc/scipy-1.5.0/reference/generated/scipy.integrate.solve_ivp.html)):
 1. The function representing the difference equation, 
 1. The start and finish times to integrate over,
 1. The initial conditions (i.e. $\mathbf{y}$ at the start time)
 
We also give it one optional 'keyword' argument *max_step*, otherwise it will work out the largest step size it can get away with and still get a reasonable solution.  This is usually too big to make nice-looking plots, but it can be useful behaviour if you only want to know the value at the finish time.

In [5]:
from scipy.integrate import solve_ivp

TIME_RANGE = [0,10]

THETA_START = np.pi/2
OMEGA_START = 0

soln = solve_ivp(part2_pend_ode, TIME_RANGE, [THETA_START, OMEGA_START], max_step = 0.05)

#Can check everything came out ok before we try to use the solution
#print(soln)

soln_times =soln.t
soln_theta, soln_omega = soln.y

#Normalise the solution to plot both on the same axis
soln_theta /= np.max(soln_theta)
soln_omega /= np.max(soln_omega)


<div class="alert alert-block alert-warning">
<h3>A cross check</h3>
</div>

Compare (visually) the SciPy library solver to yours.  Do they produce the same solutions?

<div class="alert alert-block alert-warning"></div>




## Part 3:  Bungee jumping

Let's model a more exciting situation. You are running a bungee jump company. You are almost ready to go, but you need to decide what kind of bungee cord will produce the most exciting experience -- ideally without causing too much harm to your customers. We can model the jump by considering the various forces on the jumper:
  *  Gravity $mg$,
  *  The elastic force from the cord, $E$
  *  Air resistance or drag $D$, which is a function of the jumper's velocity $v$

So in general the forces on the jumper of mass $m$ are:
$$
m \vec{a} = m \vec{g} + \vec{E} + \vec{D}
$$

Working in a co-ordinate system where $y$ is upward and we can ignore the horizontal motion of the jumper, we can write a differential equation in terms of position, velocity, and acceleration:
$$
 \ddot{y} = g + \frac{E(y)}{m} + \frac{D(\dot{y})}{m}
$$

Now we need to think about the forces.  

**Gravity** is easy. $g$ is just a constant $-9.8 \mathrm{m/s^2}$, since the definition above is for the *upward* acceleration.   

The **drag** force depends only on the speed of the jumper.  The direction of $D$ always acts in the opposite direction to the velocity $\dot{y}$, but it contains two components: laminar drag, which is proportional to velocity, and turbulent drag, which is proportional to the velocity squared.  So we write that as:

$$
D(\dot{y}) = - c_l \dot{y} - c_t |\dot{y}|\dot{y}.
$$

*Make sure you understand why we don't just write $\dot{y}^2$.* The exact values of $c_l$ and $c_t$ depend on many details about the jumper. Since this is a hypothetical exercise, setting both to $1$ is good enough.

The **elastic force** requires a bit more care.  The cord will obey Hooke's law, i.e. $E = -kx$, where $x$ is the *extension* beyond the natural length $L$.  Unlike a spring, a bungee cord does not produce any force if it is not under tension.


The bungee is attached to the platform from which the jumper leaps, this height is $y_0$. So the untensioned position of the cord end is a distance $L$ below the platform, although the jumper will come to rest a bit below this, as the cord is stretched by the jumper's weight.

The cord has a natural length of $L = 30 \mathrm{m}$ and the jump platform / attachment point is $y_0 = 80\mathrm{m}$ above the ground. At this point you might find it useful to draw a sketch of what you expect $y$ (the jumper's height above the ground) to look like, as a function of time after they jump from the platform.

<div class="alert alert-block alert-success">
    <h3>Computing task: The bungee jump</h3>
</div>

Use the techniques from the harmonic oscillator example to solve your bungee ODE using the Python's ODE solver. You might find it useful to start by working out what the functions for $E(y-y_0)$ and $D(\dot{y})$ should look like, and checking that your code produces these forms. Then sum the forces to produce a function representing the difference equation, remembering this should have the same signature as the `part2_pend_ode(t, coords)` above, and that again `coords` is a pair of values representing $(y, \dot{y})$.   

The typical range for your customers weights $80 \pm 20\, \mathrm{kg}$. Assuming $80\mathrm{kg}$, select a few reasonable values of the spring constant $k$ (it should be $\mathcal{O}(10)$) and test and plot the evolution of the system for these values. Try plotting various variables that characterise the system, i.e. the velocity, position, and forces being felt by the jumper. Make sure the graphs are clear, well-labelled, and easy to read.

There are some considerations to make when ensuring no harm comes to the prospective bungee-jumpers:

* Do they hit the ground, and how? How would you model what happens afterwards?
* What acceleration is the jumper experiencing throughout the jump?

Use your graphs to provide answers to these questions. Once you have found a suitable values of $k$, check if these still work for $60\mathrm{kg}$ and $100 \mathrm{kg}$ jumpers. 

<div class="alert alert-block alert-success"></div>


<div class="alert alert-block alert-warning">
    <h3> Bonus task: hitting the ground</h3>
</div>

It's rather macabre, so we didn't specify a component to the ODE for what to do if the jumper hits the ground. Obviously as the proprietor of a responsible hypothetical Xsports company you would never let this happen... but "Just in case", you think, "how would I include a ground impact in my model and get a physical result?".  Describe a way to implement this (it may be useful that computer functions are more flexible than mathematical functions) and if you are feeling confident, try implementing it in your code. 

<div class="alert alert-block alert-warning"> </div>
